# 6. Eine Audioanwendung (und advanced plotting)

## 1. generieren und anzeigen

In [ ]:
# pip install simpleaudio

In [ ]:
import numpy as np
import simpleaudio as sa
import matplotlib.pyplot as plt

Eine einzelne Schwingung kann über ihre Frequenz, ihre Lautstärke und ihre Dauer angegeben werden. In der Praxis setzen wir die Amplitude auf einen Standardwert (1) und müssen aber zusätzlich die Anzahl an Datenpunkten pro Sekunde angeben.

Im ersten Schritt definieren wir eine Funktion
```python
create_wave_and_timeaxis(frequency, fs, seconds)
```
deren Übergabeparameter auf 440, 44100 und 1 vorinitialisiert sind.

Es soll zwei Rückgabewerte geben:
* note = die Schwingung mit der Frequenz frequency und seconds * fs vielen Werten (Siehe hierzu: np.sin() )
und
* t = ein Array ebenfalls der Länge seconds * fs mit erstem Wert 0 und höchsten Wert seconds. (Siehe hierzu: np.linspace() )

Wobei t zuerst(!) für die Berechnung von note gebraucht wird.

In [ ]:
def create_wave_and_timeaxis(frequency=440, fs=44100, seconds=1):
    t = np.linspace(0, seconds, fs*seconds, endpoint=False)
    note = np.sin(2*np.pi*frequency*t)
    return note, t

Die Funktion play_tone(note) normalisiert zuerst die Daten, so dass der lauteste Datenpunkt auf 1 liegt und spielt das gegebene Array ab.

In [ ]:
def play_tone(note, fs=44100):

    # Ensure that highest value is in 16-bit range
    audio = note * (2**15 - 1) / np.max(np.abs(note))

    # Convert to 16-bit data
    audio = audio.astype(np.int16)
    # print(audio)

    # Start playback
    play_obj = sa.play_buffer(audio, 1, 2, fs)

    # Wait for playback to finish before exiting
    play_obj.wait_done()

Die Funktion plot_data(x, y) zeichnet einen Graphen mit den Daten des y-Arrays auf der Achse der x-Werte.

In [ ]:
def plot_data(x, y):
    plt.plot(x,y)

Fülle die Variablen note und t durch den Aufruf von create_wave_and_timeaxis()

In [ ]:
note, t = create_wave_and_timeaxis()

Spiele note ab.

In [ ]:
play_tone(note)

Erzeuge einen neuen Sinuston der eine Oktave unter dem ersten liegt.

In [ ]:
note_A3, time = create_wave_and_timeaxis(frequency=220)

In [ ]:
play_tone(note_A3)

Zeichne die Töne mit der plot Funktion.

In [ ]:
plot_data(t[0:1000], note[0:1000])

Bonus-Level: plotten geht noch schöner mit plt.subplots. So lassen sich eigentschaften wie Titel, x-Achsen-Ausschnitt (set_xlim), Farben und noch mehr einstellen. Einmal Schmöckern und Ausprobieren bitte.

In [ ]:
def plot_data_advanced(x, y, xlim=0.03):
    fig, ax = plt.subplots(facecolor = '0.9')
    ax.plot(x, y, "g", alpha=0.5)
    ax.set_xlabel("time in sec")
    ax.set_ylabel("amplitude")
    ax.set_xlim(0,xlim)
    ax.set_title("waveform of an audio snipped", fontsize = 14, fontweight = "bold")
    plt.show()

In [ ]:
plot_data_advanced(t, note, xlim = 0.05)

## 2. mit librosa und IPython.display WAV-Dateien einlesen und anzeigen

In den Modulen Ipython.display und librosa gibt es sehr viele hilfreiche Funktionen zur Darstellung, Analyse und Manipulation von Audiodaten:

In [ ]:
import librosa
import librosa.display
import IPython.display as ipd

Zuerst laden wir drei Audiofiles (.wav-Dateien) und hören sie an:

In [ ]:
piano_file = "011PFNOF60.WAV"
violin_file = "151VNNOF60.WAV"
double_bass_file = "181CBNOF60.WAV"

In [ ]:
ipd.Audio(piano_file)

In [ ]:
ipd.Audio(violin_file)

In [ ]:
ipd.Audio(double_bass_file)

Audiodateien einlesen mit librosa.

In [ ]:
piano, sr = librosa.load(piano_file) # sr = sample rate oft auch als fs
violin, _ = librosa.load(violin_file)
double_bass, _ = librosa.load(double_bass_file)

Was haben wir eigentlich genau eingelesen? Gib die "shape" aus, das ganze array und die samplerate sr.

In [ ]:
print(piano.shape)

In [ ]:
# total number of samples in audio file
tot_samples = len(piano)
tot_samples

Wie lange dauert das eingelesene "piano" File?

In [ ]:
# duration of piano audio in seconds
duration = tot_samples / sr
print("The audio lasts for",duration,"seconds")

Zeige mit 3 subplots die Instrumentefiles unter einander an.

In [ ]:
plt.figure(figsize=(15,17))

plt.subplot(3, 1, 1)
librosa.display.waveshow(piano, alpha=0.5)
plt.ylim(-1,1)
plt.title("Piano")

plt.subplot(3, 1, 2)
librosa.display.waveshow(violin, alpha=0.5)
plt.ylim(-1,1)
plt.title("Violin")

plt.subplot(3, 1, 3)
librosa.display.waveshow(double_bass, alpha=0.5)
plt.ylim(-1,1)
plt.title("Doulbe Bass")

plt.show()

### 3. Audio im Frequenzbereich

In [ ]:
def plot_magnitude_spectrum(signal, sr, title, f_ratio=1):
    X = np.fft.fft(signal)
    X_mag = np.absolute(X)

    plt.figure(figsize=(18, 5))

    f = np.linspace(0, sr, len(X_mag))
    f_bins = int(len(X_mag)*f_ratio)

    print(f[np.argmax(X_mag[:f_bins])])

    plt.plot(f[:f_bins], X_mag[:f_bins])
    plt.xlabel('Frequency (Hz)')
    plt.title(title)

In [ ]:
plot_magnitude_spectrum(piano, sr, "piano", 0.5)

In [ ]:
plot_magnitude_spectrum(violin, sr, "violin", 0.5)

In [ ]:
plot_magnitude_spectrum(double_bass, sr, "double bass", 0.5)

Um die Veränderung der Frequenzen über die Zeit zu sehen, müssen Nachbarschaften -so genannte Frames- bestimmt werden. Aus diesen wird jeweils ein Spektrum berechnet. Werden diese Nacheinander angezeigt, ergibt sich aus vielen aufeinanderfolgenden Spektren ein Spektrogram. Die Frames haben alle eine gewisse Größe (FRAME_SIZE) und, da sie überlappen, einen Versatz (HOP_SIZE).

In [ ]:
FRAME_SIZE = 2048
HOP_SIZE = 512

In [ ]:
S_scale = librosa.stft(violin, n_fft=FRAME_SIZE, hop_length=HOP_SIZE)

In [ ]:
S_scale.shape

In [ ]:
type(S_scale[0][0])

In [ ]:
Y_scale = np.abs(S_scale) ** 2

In [ ]:
def plot_spectrogram(Y, sr, hop_length, y_axis="linear"):
    plt.figure(figsize=(25, 10))
    librosa.display.specshow(Y,
                             sr=sr,
                             hop_length=hop_length,
                             x_axis="time",
                             y_axis=y_axis)
    plt.colorbar(format="%+2.f")

In [ ]:
plot_spectrogram(Y_scale, sr, HOP_SIZE)

In [ ]:
Y_log_scale = librosa.power_to_db(Y_scale)
plot_spectrogram(Y_log_scale, sr, HOP_SIZE)

In [ ]:
plot_spectrogram(Y_log_scale, sr, HOP_SIZE, y_axis="log")